In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
bangalore_auto = pd.read_csv('bangalore_pickup.csv')
bangalore_auto_hexes = bangalore_auto['pickup_location_hex_8'].unique().tolist()
print(bangalore_auto_hexes)
print(len(bangalore_auto_hexes))

['88618925cbfffff', '8861892437fffff', '8860145b55fffff', '8861892589fffff', '8860145b5dfffff', '88618925ddfffff', '88618925c7fffff', '8860145b01fffff', '88618925c1fffff', '88618925d9fffff', '88618925c3fffff', '8860145b59fffff', '8860145b09fffff', '886189258bfffff', '8861892433fffff', '8860145b43fffff', '8861892431fffff', '8860145b57fffff', '8861892435fffff', '886189258dfffff', '8860145b51fffff', '8860145b53fffff', '8860145a65fffff', '8860145b0dfffff', '8861892581fffff', '8860145b5bfffff', '8860145b0bfffff']
27


In [9]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude
from orders.order_logs_immutable
where yyyymmdd >= '20230417'
and yyyymmdd <= '20230423'
and service_obj_service_name = 'Link'
and event_type in ('started')
and city_name = 'Bangalore'
and pickup_location_hex_8 in ('88618925cbfffff', '8861892437fffff', '8860145b55fffff', '8861892589fffff', '8860145b5dfffff', '88618925ddfffff', '88618925c7fffff',
 '8860145b01fffff', '88618925c1fffff', '88618925d9fffff', '88618925c3fffff', '8860145b59fffff', '8860145b09fffff', '886189258bfffff', '8861892433fffff',
  '8860145b43fffff', '8861892431fffff', '8860145b57fffff', '8861892435fffff', '886189258dfffff', '8860145b51fffff', '8860145b53fffff', '8860145a65fffff',
   '8860145b0dfffff', '8861892581fffff', '8860145b5bfffff', '8860145b0bfffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [10]:
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude
0,6442b0fc2a803956b68c110b,0.838,88618925c3fffff,12.929564,77.608739,Bangalore,88618925c3fffff,88618925c1fffff,12.924229,77.618111,12.929536,77.608734,212124,21,2115,Link,21:00,5,20230421,started,None,1682092520486,started,64133068b6945d763d8a3bcf,None,88618925c3fffff,12.929542,77.608720
1,6442acf0f23f90544e3d37c4,2.402,886189258bfffff,12.932508,77.600610,Bangalore,8861892437fffff,8861892ebbfffff,12.997777,77.614795,12.910215,77.614326,210408,21,2100,Link,21:00,5,20230421,started,None,1682091736329,started,63d8fb2ad330c079894f3b83,None,886189258bfffff,12.930666,77.600515
2,6442b03144636e16f07f8f02,2.552,88618925ddfffff,12.918011,77.605347,Bangalore,88618925abfffff,88618925abfffff,12.959608,77.601756,12.959666,77.601830,211801,21,2115,Link,21:00,5,20230421,started,None,1682092599968,started,63d634afb5f44286bc0d0640,None,88618925ddfffff,12.918177,77.605497
3,6442ad1e5028ee299f996771,0.129,8861892589fffff,12.932789,77.609406,Bangalore,8861892589fffff,8860145b55fffff,12.980718,77.574680,12.932783,77.609421,210454,21,2100,Link,21:00,5,20230421,started,None,1682091698234,started,5cd5b19f377155163cf69f4a,None,8861892589fffff,12.932738,77.609493
4,6442b26e24f8d87437f68110,0.559,8860145b55fffff,12.976412,77.571034,Bangalore,8860145b55fffff,8860145a45fffff,12.936159,77.561256,12.977534,77.571114,212734,21,2115,Link,21:00,5,20230421,started,None,1682092936045,started,5e62587a6ad99b58025eea7a,None,8860145b55fffff,12.977435,77.570923


In [11]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Bangalore')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [12]:
#Add drop, pickup, captain
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df_apr = df_apr.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df_apr = df_apr.rename(columns = {'cluster':'pickup_cluster'})
df_apr = df_apr.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df_apr = df_apr.rename(columns = {'cluster':'drop_cluster'})
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,pickup_cluster,drop_cluster
0,6442b0fc2a803956b68c110b,0.838,88618925c3fffff,12.929564,77.608739,Bangalore,88618925c3fffff,88618925c1fffff,12.924229,77.618111,12.929536,77.608734,212124,21,2115,Link,21:00,5,20230421,started,None,1682092520486,started,64133068b6945d763d8a3bcf,None,88618925c3fffff,12.929542,77.608720,Tavarakere,Madiwala
1,6442acf0f23f90544e3d37c4,2.402,886189258bfffff,12.932508,77.600610,Bangalore,8861892437fffff,8861892ebbfffff,12.997777,77.614795,12.910215,77.614326,210408,21,2100,Link,21:00,5,20230421,started,None,1682091736329,started,63d8fb2ad330c079894f3b83,None,886189258bfffff,12.930666,77.600515,Adugodi,Frazer Town
2,6442b03144636e16f07f8f02,2.552,88618925ddfffff,12.918011,77.605347,Bangalore,88618925abfffff,88618925abfffff,12.959608,77.601756,12.959666,77.601830,211801,21,2115,Link,21:00,5,20230421,started,None,1682092599968,started,63d634afb5f44286bc0d0640,None,88618925ddfffff,12.918177,77.605497,BTM,Shanti Nagar
3,6442ad1e5028ee299f996771,0.129,8861892589fffff,12.932789,77.609406,Bangalore,8861892589fffff,8860145b55fffff,12.980718,77.574680,12.932783,77.609421,210454,21,2100,Link,21:00,5,20230421,started,None,1682091698234,started,5cd5b19f377155163cf69f4a,None,8861892589fffff,12.932738,77.609493,Tavarakere,Majestic
4,6442b26e24f8d87437f68110,0.559,8860145b55fffff,12.976412,77.571034,Bangalore,8860145b55fffff,8860145a45fffff,12.936159,77.561256,12.977534,77.571114,212734,21,2115,Link,21:00,5,20230421,started,None,1682092936045,started,5e62587a6ad99b58025eea7a,None,8860145b55fffff,12.977435,77.570923,Majestic,Basavanagudi


In [13]:
df_apr.to_csv('link_pickup_points.csv', index = False)